In [111]:
import Pkg

Pkg.add("JLD2")

    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed FileIO ─ v1.16.3
   Installed JLD2 ─── v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [033835bb] + JLD2 v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [5789e2e9] + FileIO v1.16.3
  [033835bb] + JLD2 v0.4.51
Precompiling project...
  ✓ FileIO
  ✓ Plots → FileIOExt
  ✓ JLD2
  ✓ TaylorSeries → TaylorSeriesJLD2Ext
  4 dependencies successfully precompiled in 79 seconds. 538 already precompiled.


In [2]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR
using JLD2

In [3]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [5]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,60, 100_000)
returns_scenarios = scenarios[1:3,:,:]

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
 -0.0255912    0.0153249   0.0410305  …   0.0134946    0.114737
 -0.0709722    0.00215142  0.0183902     -0.0687901   -0.0481577
  0.000767663  0.0128744   0.0112964      0.00286056  -0.00729518

[:, :, 3] =
 -0.0177534   0.0542137    0.174041    …  -0.0279776   0.0411065    0.141107
  0.0544895   0.00449311  -0.0264064      -0.0694363   0.0399175   -0.0777425
  0.0113094  -0.00220942  -0.00406126     -0.00393851  0.00929861  -0.0013118

;;; … 

[:, :, 59] =
  0.0994771   0.0799027   -0.0514079  …  0.091565    -0.00624314   0.124097
 -0.0307044   0.00107344   0.0432197     0.06081     -0.010537    -0.0376569
 -0.00274682  0.00396313   0.0052367     0.00625212  -0.00743294  -0.0156492

[:, :, 60] =
 0.0566079   0.0349726   0.0848

In [4]:
includet("kmeans_moments_lattice.jl")
using .LatticeGeneration

In [20]:
returns_scenarios_ann = zeros(3,100_000,15)
for asset in 1:3
    returns_scenarios_ann[asset,:,:] = exp.(annualise(returns_scenarios[asset,:,2:end],4))
end
returns_scenarios_ann


3×100000×15 Array{Float64, 3}:
[:, :, 1] =
 1.03814   1.18003   1.32981   1.04836   …  0.991326  0.870595  1.25006
 0.996445  0.872951  0.977248  0.878046     0.835413  0.965795  0.877362
 1.00683   0.997856  1.02917   0.98942      0.992249  1.04286   1.00147

[:, :, 2] =
 1.14638   0.991848  1.0613   1.04102  …  1.18514  0.987272  1.15997  1.02208
 0.930223  1.02039   1.00128  1.01483     0.96673  1.17105   1.10276  1.03834
 0.992769  1.03773   1.00921  1.01704     1.01402  1.03889   1.02351  1.06016

[:, :, 3] =
 1.03956   1.35075   0.904098  …  1.06175  0.973323  1.311    1.11439
 0.939936  0.980319  1.08041      1.0025   1.11333   1.14061  1.01156
 1.02103   1.01924   1.00518      1.0195   1.01805   1.02293  1.05099

;;; … 

[:, :, 13] =
 1.25055   1.12013  1.00907   0.875529  …  0.975677  1.03013   1.0418
 0.935895  1.11704  0.962021  1.20381      1.18953   0.891497  0.94771
 1.00503   1.00992  1.00277   0.992363     0.99245   0.976582  0.971182

[:, :, 14] =
 0.736483  1.00663  0

In [22]:
res = generate_lattice(returns_scenarios_ann, 17,15)


Lattice([1.0592345906772338 1.465012213163772 … 1.2626254364258738 1.0384896639025631; 1.2824012019722257 1.3209148024880717 … 1.0951995497952882 1.2960022821913877; … ; 1.24297289583853 1.4353528875763193 … 1.0878430868351783 1.109912342050847; 1.1080030398593914 1.2745261503138794 … 0.8966011450716054 0.9287604610706303;;; 0.887040753784256 0.982186473729422 … 0.9863207155344317 0.9605722735936422; 1.0274664166129501 0.8996924085193001 … 1.1704258333166766 1.0526752406102102; … ; 1.095784045056427 0.9721185588871997 … 1.1251939667165438 0.9920778205762303; 0.8812707955810356 1.082746466751104 … 1.0665082211413062 0.9528083068465363;;; 1.0048932300341245 1.015836978685782 … 0.9992000318538323 0.9914392779297038; 1.0083837362993968 0.9891211656842298 … 1.0459435602667502 1.0345218166687264; … ; 1.032967063017141 1.0077920726566698 … 1.0056558248728613 0.9727380532909757; 0.9558181287417183 1.0230447326464345 … 1.0189427655670962 1.0020927656322343], [[1.0;;], [0.06534 0.04487 … 0.07863

In [16]:
round.(res.states[3,:,:]; digits=4)

17×3 Matrix{Float64}:
 1.1725  1.0626  1.0155
 1.3335  0.9778  1.0669
 1.6304  0.8731  0.9505
 1.1129  1.3585  1.1022
 1.312   1.2247  1.0302
 1.2498  0.9968  0.9544
 1.3683  0.8768  1.0045
 1.0251  1.1631  0.9631
 0.8863  0.98    0.9413
 1.0343  1.1092  1.0819
 1.9338  0.9734  1.0338
 1.0113  0.9124  1.0059
 1.534   1.0625  1.0036
 0.8642  1.0737  1.013
 1.1271  0.834   0.9259
 0.9199  1.2815  1.027
 1.5533  1.1565  1.0969

In [17]:
round.(res.states[4,:,:]; digits=4)

17×3 Matrix{Float64}:
 1.9892  0.9473  1.0052
 1.463   0.9219  1.0309
 1.5355  0.8595  0.9457
 1.1165  0.8342  0.9081
 1.197   1.252   1.1162
 0.8859  0.9705  0.9323
 1.1603  1.0251  1.0725
 0.9529  1.1732  0.967
 1.3074  1.2126  1.0221
 1.1988  1.0475  1.0056
 1.6474  1.12    1.0775
 1.5419  1.0618  0.9832
 1.2222  1.0154  0.9418
 0.8999  0.9948  1.0033
 1.1436  0.8744  0.987
 0.991   1.3786  1.0438
 0.9309  1.156   1.0435

In [23]:
round.(res.probs[4][1,:]; digits=2)

17-element Vector{Float64}:
 0.04
 0.1
 0.09
 0.01
 0.1
 0.04
 0.06
 0.05
 0.08
 0.02
 0.07
 0.06
 0.02
 0.06
 0.09
 0.02
 0.07

In [24]:
save_object("scenario_grid.jld2", res)

In [60]:
grid = load_object("scenario_grid.jld2")

Lattice([1.0853933315390984 1.218288162475466 … 1.8355566846831484 1.4009515471553637; 0.94539648550184 0.9352273412884807 … 1.5265792998503565 1.2383234954409708; … ; 0.9191554431977321 1.154816879403226 … 1.3300414893432977 1.038763668706857; 0.9261635028461954 1.7917767202361508 … 1.1708527976718917 1.3322260375257169;;; 0.9765946310583116 1.1774415120338133 … 0.9291402188272971 0.8899189392863953; 1.2181816749731287 1.0059295736387661 … 1.0865583310648845 0.8873290097745962; … ; 1.1156717683855484 0.9243580513031538 … 1.100655126873907 1.0823987352357525; 1.109849303328651 0.9550545807263788 … 0.9243075236913665 1.0942922799016646;;; 1.0402671160866452 1.0463306482683974 … 1.0228155547306848 1.0562637488834923; 1.0101162372306656 0.9790255920467821 … 1.0822322232049224 0.9539788581889932; … ; 0.9764644843180704 1.0088132672878474 … 0.9857306066451239 1.0572844235140704; 0.9713402918750139 1.0025639047032668 … 1.0070728088493606 0.9795746837072669], [[1.0;;], [0.157 0.10318 … 0.0706

In [38]:
using  Test
@test grid.states[5,:,:] ≈ res.states[5,:,:]

Test Passed

In [41]:
res.states[3,:,3]

9-element Vector{Float64}:
 1.0143305473204354
 0.9854785572603496
 1.003604484079091
 1.0038924296581662
 1.0107697123646282
 0.9910509301591774
 1.0332650815623272
 1.0259226703370474
 1.034748568861909

scenarios_moments

t | mean | std | var | skewness| kurtosis | 
--- | --- | --- | --- | --- | --- |

print moments

lattice_moments

#### TODO

- evaluate moments